In [4]:
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt
from transformers import pipeline
from sklearn.model_selection import train_test_split, cross_val_score,KFold, cross_val_predict, GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier


In [5]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

## Download All Policies - Train Set

In [6]:
##Privacy Policies download:
websites = ['Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson']

sequence_to_classify = []

for i in websites:
    try:
        with open((i+".txt"), "r") as f:
            i = f.readlines()
            sequence_to_classify.append(i)
    except:
        with open((i+".txt"), "r", encoding='cp1252') as f:
            i = f.readlines()
            sequence_to_classify.append(i)

### Train Set Vectors:

In [7]:
data_collection_train = [[1,2,3,4,5,6,7,8],[1,2,3,4,5,6],[1,3,5,6,7],[1,3,7],[1,2,3,4,6,7]
                         ,[1,2,3,4,5,6,7],[1,4,5,6,7],[1,3,5,6,7],[1,2,3,5,6,7],[1,2,3,4,5,6,7]
                         ,[1,2,3,5,6,7],[1,3,5,6,7],[1,3,7],[1,2,3,7],[1,3,7]
                         ,[1,2,5,6],[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[1,3,5,7],[1,2,3,6]
                         ,[1,3,6,7],[1,2,3,5,6,7,8],[1,2,3,4,5,6,7],[1,3,5,6,7],[1,3,5,6,7,8]
                         ,[1,3,5,6,7],[1,2,3,4,5,6,7,8],[1,2,3,4,5,6,7],[1,3,5,6],[1,2,3,5,6,7,8]
                         ,[1,2,3,5,6,7,8],[1,2,3,4,5,6,7,8],[1,2,3,5,6,7,8],[1,2,3,4,5,6,7],[1,3,4,5,6,7]
                         ,[1,3,5,7],[1,2,3,4,5,6,7,8],[1,3,7],[1,2,3,4,5,6,7,8],[1,2,3,4,5,6,7]
                         ,[1,2,3,5,7,8],[1,3,7],[1,3,5,7,8],[1,3,4,5,6,7,8],[1,2,3,6,7]
                         ,[1,3,5,6,7,8],[1,2,3,4,5,6,7],[1,3,5,6,7,8],[1,3,5,6,7]]

In [8]:
data_collection_train_personal = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
data_collection_train_media = [1,1,0,0,1,1,0,0,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0,1,0,1,0,0]
data_collection_train_browser = [1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
data_collection_train_voice = [1,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,1,1,0,1,0,1,1,0,0,0,1,0,0,1,0,0]
data_collection_train_payments = [1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,0,1,1,1,1]
data_collection_train_location = [1,1,1,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1]
data_collection_train_IP = [1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
data_collection_train_passward = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,1,1,0,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0]

### Data Security Parameter

In [9]:
##data_security Matrix - _personal [1]:

Phrases = ['Privacy Policy'
           ,'we save your name, address, phone number, birth date, or email address'
           ,'we are not collect your name, address, phone number, birth date, and email address'
           ,'we collect your personal data'
           ,'we do not collect your personal data']

##Matrix creation:
matrix_data_collection_ML_personal = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_personal['Privacy Policy'] = websites
matrix_data_collection_ML_personal.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_personal:
        matrix_data_collection_ML_personal.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1

matrix_data_collection_ML_personal

,"we save your name, address, phone number, birth date, or email address","we are not collect your name, address, phone number, birth date, and email address",we collect your personal data,we do not collect your personal data
Privacy Policy,,,,
Google,0.281912,0.302289,0.981916,0.267769
Aws,0.260433,0.25161,0.953546,0.732466
AliExpress,0.292289,0.325571,0.86903,0.147927
Meta,0.311984,0.311889,0.982988,0.906464
TikTok,0.268777,0.289224,0.892116,0.755127
YouTube,0.298692,0.350057,0.946936,0.372887
Waze,0.37765,0.376907,0.934453,0.249633
Wix,0.298922,0.298791,0.977923,0.221816
Bookings,0.321984,0.312714,0.990194,0.798815


### Test The Model - ML - Personal Data

In [12]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_personal.iloc[:35,:] 
X_test = matrix_data_collection_ML_personal.iloc[34:,:]
y_train = data_collection_train_personal[:35]
y_test = data_collection_train_personal[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

# ##Logistic Reg Classifier
# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# pred_clf = clf.predict(X_test)
# print('Logistic Reg Classifier:', pred_clf)
# print(classification_report(y_test, pred_clf))

# ##SVM
# SVM = SVC(kernel='linear')
# SVM = SVM.fit(X_train, y_train)
# pred_SVM = SVM.predict(X_test)
# print('SVM Classifier:', pred_SVM)
# print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

KNN Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

neural_network

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [13]:
pred_personal = pred_tree

In [14]:
##data_security Matrix - _media [2]:

Phrases = ['Privacy Policy'
           ,'we save your media'
           ,'we are not collect your media'
           ,'we collect your photos and images'
           ,'we are not saving your photos and images']

##Matrix creation:
matrix_data_collection_ML_media = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_media['Privacy Policy'] = websites
matrix_data_collection_ML_media.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_media:
        matrix_data_collection_ML_media.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collection_ML_media

,we save your media,we are not collect your media,we collect your photos and images,we are not saving your photos and images
Privacy Policy,,,,
Google,0.937889,0.637209,0.955504,0.146648
Aws,0.909985,0.863607,0.778217,0.398459
AliExpress,0.685809,0.221852,0.777231,0.044111
Meta,0.954075,0.904198,0.867779,0.367086
TikTok,0.863993,0.707232,0.877363,0.344582
YouTube,0.923091,0.419085,0.902374,0.149081
Waze,0.835655,0.526477,0.59618,0.044798
Wix,0.836957,0.635409,0.805327,0.072721
Bookings,0.917105,0.883325,0.909622,0.106977


In [15]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_media.iloc[:35,:] 
X_test = matrix_data_collection_ML_media.iloc[34:,:]
y_train = data_collection_train_media[:35]
y_test = data_collection_train_media[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [0 0 2 0 1 1 0 1 0 2 0 1 1 0 0]
              precision    recall  f1-score   support

           0       0.75      0.67      0.71         9
           1       0.00      0.00      0.00         0
           2       0.50      0.17      0.25         6

    accuracy                           0.47        15
   macro avg       0.42      0.28      0.32        15
weighted avg       0.65      0.47      0.52        15

Random Forest Classifaier: [0 0 2 1 1 0 0 1 0 2 0 1 0 0 0]
              precision    recall  f1-score   support

           0       0.56      0.56      0.56         9
           1       0.00      0.00      0.00         0
           2       0.50      0.17      0.25         6

    accuracy                           0.40        15
   macro avg       0.35      0.24      0.27        15
weighted avg       0.53      0.40      0.43        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

       

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/pyth

neural_network Classifier: [1 0 2 0 1 0 0 1 1 0 1 1 0 0 1]
              precision    recall  f1-score   support

           0       0.57      0.44      0.50         9
           1       0.00      0.00      0.00         0
           2       1.00      0.17      0.29         6

    accuracy                           0.33        15
   macro avg       0.52      0.20      0.26        15
weighted avg       0.74      0.33      0.41        15



/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetri

In [16]:
pred_media = pred_tree

In [17]:
##data_security Matrix - _browser [3]:

Phrases = ['Privacy Policy'
           ,'we save your browsing history'
           ,'we save your browsing preferences'
           ,'we are not collect your browsing history'
           ,'we are not collect your browsing preferences']

##Matrix creation:
matrix_data_collection_ML_browser = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_browser['Privacy Policy'] = websites
matrix_data_collection_ML_browser.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_browser:
        matrix_data_collection_ML_browser.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collection_ML_browser

,we save your browsing history,we save your browsing preferences,we are not collect your browsing history,we are not collect your browsing preferences
Privacy Policy,,,,
Google,0.844826,0.886259,0.385589,0.591475
Aws,0.906165,0.908969,0.805677,0.818994
AliExpress,0.737969,0.682618,0.179964,0.166785
Meta,0.946151,0.931737,0.91018,0.89538
TikTok,0.778982,0.816761,0.645819,0.690651
YouTube,0.896308,0.897105,0.481192,0.497081
Waze,0.728593,0.781011,0.325879,0.396393
Wix,0.711233,0.594068,0.381705,0.354922
Bookings,0.921572,0.922506,0.627632,0.714347


In [18]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_browser.iloc[:35,:] 
X_test = matrix_data_collection_ML_browser.iloc[34:,:]
y_train = data_collection_train_browser[:35]
y_test = data_collection_train_browser[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 1 0 1 1 0 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.87      0.93        15

    accuracy                           0.87        15
   macro avg       0.50      0.43      0.46        15
weighted avg       1.00      0.87      0.93        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weight

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/pyth

In [19]:
pred_browser = pred_forest

In [20]:
##data_security Matrix - data_collection_train_voice [4]:

Phrases = ['Privacy Policy'
           ,'we save your recording'
           ,'we save your audio'
           ,'we are collecting your voice messages'
           ,'we are not collect your audio and recording']

##Matrix creation:
matrix_data_collection_ML_voice = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_voice['Privacy Policy'] = websites
matrix_data_collection_ML_voice.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_voice:
        matrix_data_collection_ML_voice.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collection_ML_voice

,we save your recording,we save your audio,we are collecting your voice messages,we are not collect your audio and recording
Privacy Policy,,,,
Google,0.941591,0.960918,0.94851,0.280658
Aws,0.909562,0.875818,0.824499,0.681671
AliExpress,0.784952,0.737044,0.642609,0.208739
Meta,0.973421,0.93645,0.850466,0.809944
TikTok,0.881189,0.836365,0.731347,0.427886
YouTube,0.93815,0.924631,0.907074,0.19376
Waze,0.868342,0.841272,0.559361,0.200665
Wix,0.807548,0.85433,0.893022,0.336657
Bookings,0.926708,0.939017,0.948596,0.308483


In [21]:
# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_voice.iloc[:35,:] 
X_test = matrix_data_collection_ML_voice.iloc[34:,:]
y_train = data_collection_train_voice[:35]
y_test = data_collection_train_voice[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 0 0 0 1 0 0 1 1 1 0 1 1 1 1]
              precision    recall  f1-score   support

           0       0.67      0.44      0.53         9
           1       0.44      0.67      0.53         6

    accuracy                           0.53        15
   macro avg       0.56      0.56      0.53        15
weighted avg       0.58      0.53      0.53        15

Random Forest Classifaier: [1 0 0 0 1 0 0 1 0 1 0 0 1 1 0]
              precision    recall  f1-score   support

           0       0.78      0.78      0.78         9
           1       0.67      0.67      0.67         6

    accuracy                           0.73        15
   macro avg       0.72      0.72      0.72        15
weighted avg       0.73      0.73      0.73        15

Logistic Reg Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         9
           1       0.00      0.00      0.00         6

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [22]:
pred_voice = pred_forest

In [23]:
##data_security Matrix - data_collection_train_payments [5]:

Phrases = ['Privacy Policy'
           ,'we save your payments data'
           ,'we collect your payments details'
           ,'we are not collect your payments data']

##Matrix creation:
matrix_data_collecting_ML_payments = pd.DataFrame(columns = Phrases)
matrix_data_collecting_ML_payments['Privacy Policy'] = websites
matrix_data_collecting_ML_payments.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collecting_ML_payments:
        matrix_data_collecting_ML_payments.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collecting_ML_payments

,we save your payments data,we collect your payments details,we are not collect your payments data
Privacy Policy,,,
Google,0.760889,0.957064,0.35989
Aws,0.863534,0.94285,0.681946
AliExpress,0.620869,0.756453,0.14762
Meta,0.904471,0.946495,0.763563
TikTok,0.719862,0.762308,0.519818
YouTube,0.832188,0.899888,0.362435
Waze,0.629844,0.78542,0.200484
Wix,0.565954,0.901089,0.219454
Bookings,0.937423,0.985275,0.739429


In [24]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collecting_ML_payments.iloc[:35,:] 
X_test = matrix_data_collecting_ML_payments.iloc[34:,:]
y_train = data_collection_train_payments[:35]
y_test = data_collection_train_payments[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 0 1 1 1 1 1 1 1 0 0 1 0]
              precision    recall  f1-score   support

           0       0.25      0.33      0.29         3
           1       0.82      0.75      0.78        12

    accuracy                           0.67        15
   macro avg       0.53      0.54      0.53        15
weighted avg       0.70      0.67      0.68        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.79      0.92      0.85        12

    accuracy                           0.73        15
   macro avg       0.39      0.46      0.42        15
weighted avg       0.63      0.73      0.68        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.80      1.00      0.89        12

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [25]:
pred_payments = pred_neural_network

In [26]:
##data_security Matrix - data_collection_train_location [6]:

Phrases = ['Privacy Policy'
           ,'we save your location'
           ,'we are not collect your location'
           ,'we save your language'
           ,'we are not collect your language']

##Matrix creation:
matrix_data_collection_ML_location = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_location['Privacy Policy'] = websites
matrix_data_collection_ML_location.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_location:
        matrix_data_collection_ML_location.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collection_ML_location

,we save your location,we are not collect your location,we save your language,we are not collect your language
Privacy Policy,,,,
Google,0.908582,0.572177,0.948124,0.699031
Aws,0.910269,0.86503,0.911911,0.848358
AliExpress,0.746214,0.27863,0.669556,0.185862
Meta,0.94689,0.879795,0.957687,0.839334
TikTok,0.831156,0.667386,0.865321,0.673416
YouTube,0.893129,0.410609,0.942237,0.456291
Waze,0.867089,0.654738,0.873169,0.578474
Wix,0.751416,0.43633,0.816262,0.628176
Bookings,0.90146,0.86196,0.956732,0.916293


In [27]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_location.iloc[:35,:] 
X_test = matrix_data_collection_ML_location.iloc[34:,:]
y_train = data_collection_train_location[:35]
y_test = data_collection_train_location[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 1 1 0 1 1 1 0 1 0 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.58      0.70      0.64        10

    accuracy                           0.47        15
   macro avg       0.29      0.35      0.32        15
weighted avg       0.39      0.47      0.42        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.67      1.00      0.80        10

    accuracy                           0.67        15
   macro avg       0.33      0.50      0.40        15
weighted avg       0.44      0.67      0.53        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.67      1.00      0.80        10

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [28]:
pred_location = pred_neural_network

In [29]:
##data_security Matrix - data_collection_train_IP [7]:

Phrases = ['Privacy Policy'
           ,'we save your IP address'
           ,'we are not collect your browser and device details'
           ,'we save your browser and device details'
           ,'we are not collect your IP address']

##Matrix creation:
matrix_data_collection_ML_IP = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_IP['Privacy Policy'] = websites
matrix_data_collection_ML_IP.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_IP:
        matrix_data_collection_ML_IP.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collection_ML_IP

,we save your IP address,we are not collect your browser and device details,we save your browser and device details,we are not collect your IP address
Privacy Policy,,,,
Google,0.927285,0.399406,0.958664,0.730843
Aws,0.864911,0.811091,0.941885,0.669195
AliExpress,0.794322,0.274946,0.834416,0.273507
Meta,0.95552,0.915104,0.971475,0.951077
TikTok,0.824655,0.523605,0.824444,0.719136
YouTube,0.923553,0.215291,0.942502,0.566714
Waze,0.826572,0.325615,0.909045,0.369936
Wix,0.829918,0.408896,0.879059,0.527915
Bookings,0.959671,0.684412,0.988619,0.860105


In [30]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_IP.iloc[:35,:] 
X_test = matrix_data_collection_ML_IP.iloc[34:,:]
y_train = data_collection_train_IP[:35]
y_test = data_collection_train_IP[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 1 0 1 1 1 1 1 1 0 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.87      0.93        15

    accuracy                           0.87        15
   macro avg       0.50      0.43      0.46        15
weighted avg       1.00      0.87      0.93        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weight

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/pyth

In [31]:
pred_IP = pred_forest

In [32]:
##data_security Matrix - data_collection_train_passward [8]:

Phrases = ['Privacy Policy'
           ,'we save your passwords'
           ,'we are not collect your passwords']

##Matrix creation:
matrix_data_collection_ML_passward = pd.DataFrame(columns = Phrases)
matrix_data_collection_ML_passward['Privacy Policy'] = websites
matrix_data_collection_ML_passward.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_data_collection_ML_passward:
        matrix_data_collection_ML_passward.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_data_collection_ML_passward

,we save your passwords,we are not collect your passwords
Privacy Policy,,
Google,0.803548,0.392315
Aws,0.607057,0.392887
AliExpress,0.158173,0.073476
Meta,0.825558,0.588022
TikTok,0.749715,0.585331
YouTube,0.759953,0.23613
Waze,0.46796,0.274605
Wix,0.362792,0.231374
Bookings,0.655465,0.648127


In [33]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_data_collection_ML_passward.iloc[:35,:] 
X_test = matrix_data_collection_ML_passward.iloc[34:,:]
y_train = data_collection_train_passward[:35]
y_test = data_collection_train_passward[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [0 0 0 1 0 1 0 0 0 0 0 0 1 0 0]
              precision    recall  f1-score   support

           0       0.42      0.62      0.50         8
           1       0.00      0.00      0.00         7

    accuracy                           0.33        15
   macro avg       0.21      0.31      0.25        15
weighted avg       0.22      0.33      0.27        15

Random Forest Classifaier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.53      1.00      0.70         8
           1       0.00      0.00      0.00         7

    accuracy                           0.53        15
   macro avg       0.27      0.50      0.35        15
weighted avg       0.28      0.53      0.37        15

Logistic Reg Classifier: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.53      1.00      0.70         8
           1       0.00      0.00      0.00         7

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

In [34]:
pred_passward = pred_neural_network

In [35]:
#Data Collection Parameter:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in data_collection_train_personal[34:]:
    if pred_personal[index]==1 and data_collection_train_personal[index+34]==1:
        TP +=1
    elif pred_personal[index]==0 and data_collection_train_personal[index+34]==0:
        TN +=1
    elif pred_personal[index]==0 and data_collection_train_personal[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_collection_train_media[34:]:
    if pred_media[index]==1 and data_collection_train_media[index+34]==1:
        TP +=1
    elif pred_media[index]==0 and data_collection_train_media[index+34]==0:
        TN +=1
    elif pred_media[index]==0 and data_collection_train_media[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_collection_train_browser[34:]:
    if pred_browser[index]==1 and data_collection_train_browser[index+34]==1:
        TP +=1
    elif pred_browser[index]==0 and data_collection_train_browser[index+34]==0:
        TN +=1
    elif pred_browser[index]==0 and data_collection_train_browser[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_voice[34:]:
    if pred_voice[index]==1 and data_collection_train_voice[index+34]==1:
        TP +=1
    elif pred_voice[index]==0 and data_collection_train_voice[index+34]==0:
        TN +=1
    elif pred_voice[index]==0 and data_collection_train_voice[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0   
for i in data_collection_train_payments[34:]:
    if pred_payments[index]==1 and data_collection_train_payments[index+34]==1:
        TP +=1
    elif pred_payments[index]==0 and data_collection_train_payments[index+34]==0:
        TN +=1
    elif pred_payments[index]==0 and data_collection_train_payments[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_location[34:]:
    if pred_location[index]==1 and data_collection_train_location[index+34]==1:
        TP +=1
    elif pred_location[index]==0 and data_collection_train_location[index+34]==0:
        TN +=1
    elif pred_location[index]==0 and data_collection_train_location[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_IP[34:]:
    if pred_IP[index]==1 and data_collection_train_IP[index+34]==1:
        TP +=1
    elif pred_IP[index]==0 and data_collection_train_IP[index+34]==0:
        TN +=1
    elif pred_IP[index]==0 and data_collection_train_IP[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_passward[34:]:
    if pred_passward[index]==1 and data_collection_train_passward[index+34]==1:
        TP +=1
    elif pred_passward[index]==0 and data_collection_train_passward[index+34]==0:
        TN +=1
    elif pred_passward[index]==0 and data_collection_train_passward[index+34]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 0.8875
Precision: 0.7888888888888889
Accuracy: 0.7666666666666667
F1 Score: 0.8352941176470589


In [36]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_personal = matrix_data_collection_ML_personal.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_personal.loc['Median'] =  matrix_data_collection_Median_personal.median()
matrix_data_collection_Median_personal.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_personal = []
websites_index = 0

for i in matrix_data_collection_Median_personal:
    num = matrix_data_collection_Median_personal[websites[websites_index]].loc['Median']
    if num >0.2:
        data_collection_train_pred_Median_personal.append(1)
    else:
        data_collection_train_pred_Median_personal.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_personal)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [51]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_media = matrix_data_collection_ML_media.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_media.loc['Median'] =  matrix_data_collection_Median_media.median()
matrix_data_collection_Median_media.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_media = []
websites_index = 0

for i in matrix_data_collection_Median_media:
    num = matrix_data_collection_Median_media[websites[websites_index]].loc['Median']
    if num >0.5:
        data_collection_train_pred_Median_media.append(1)
    else:
        data_collection_train_pred_Median_media.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_media)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


In [53]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_browser = matrix_data_collection_ML_browser.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_browser.loc['Median'] =  matrix_data_collection_Median_browser.median()
matrix_data_collection_Median_browser.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_browser = []
websites_index = 0

for i in matrix_data_collection_Median_browser:
    num = matrix_data_collection_Median_browser[websites[websites_index]].loc['Median']
    if num >0.5:
        data_collection_train_pred_Median_browser.append(1)
    else:
        data_collection_train_pred_Median_browser.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_browser)

[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


In [55]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_voice = matrix_data_collection_ML_voice.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_voice.loc['Median'] =  matrix_data_collection_Median_voice.median()
matrix_data_collection_Median_voice.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_voice = []
websites_index = 0

for i in matrix_data_collection_Median_voice:
    num = matrix_data_collection_Median_voice[websites[websites_index]].loc['Median']
    if num >0.7:
        data_collection_train_pred_Median_voice.append(1)
    else:
        data_collection_train_pred_Median_voice.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_voice)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


In [57]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_payments = matrix_data_collecting_ML_payments.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_payments.loc['Median'] =  matrix_data_collection_Median_payments.median()
matrix_data_collection_Median_payments.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_payments = []
websites_index = 0

for i in matrix_data_collection_Median_payments:
    num = matrix_data_collection_Median_payments[websites[websites_index]].loc['Median']
    if num >0.6:
        data_collection_train_pred_Median_payments.append(1)
    else:
        data_collection_train_pred_Median_payments.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_payments)

[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


In [59]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_location = matrix_data_collection_ML_location.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_location.loc['Median'] =  matrix_data_collection_Median_location.median()
matrix_data_collection_Median_location.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_location = []
websites_index = 0

for i in matrix_data_collection_Median_location:
    num = matrix_data_collection_Median_location[websites[websites_index]].loc['Median']
    if num >0.5:
        data_collection_train_pred_Median_location.append(1)
    else:
        data_collection_train_pred_Median_location.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_location)

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


In [61]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_IP = matrix_data_collection_ML_IP.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_IP.loc['Median'] =  matrix_data_collection_Median_IP.median()
matrix_data_collection_Median_IP.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_IP = []
websites_index = 0

for i in matrix_data_collection_Median_IP:
    num = matrix_data_collection_Median_IP[websites[websites_index]].loc['Median']
    if num >0.6:
        data_collection_train_pred_Median_IP.append(1)
    else:
        data_collection_train_pred_Median_IP.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_IP)

[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]


In [63]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_data_collection_Median_passward = matrix_data_collection_ML_passward.transpose()
    
#add the avg and median rows:
matrix_data_collection_Median_passward.loc['Median'] =  matrix_data_collection_Median_passward.median()
matrix_data_collection_Median_passward.loc['Median']['Privacy Policy'] = 'Median'

data_collection_train_pred_Median_passward = []
websites_index = 0

for i in matrix_data_collection_Median_passward:
    num = matrix_data_collection_Median_passward[websites[websites_index]].loc['Median']
    if num >0.7:
        data_collection_train_pred_Median_passward.append(1)
    else:
        data_collection_train_pred_Median_passward.append(0)
    websites_index += 1
 
print(data_collection_train_pred_Median_passward)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [65]:
#Data Collection Parameter:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in data_collection_train_pred_Median_personal:
    if data_collection_train_pred_Median_personal[index]==1 and data_collection_train_personal[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_personal[index]==0 and data_collection_train_personal[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_personal[index]==0 and data_collection_train_personal[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_collection_train_pred_Median_media:
    if data_collection_train_pred_Median_media[index]==1 and data_collection_train_media[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_media[index]==0 and data_collection_train_media[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_media[index]==0 and data_collection_train_media[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_collection_train_pred_Median_browser:
    if data_collection_train_pred_Median_browser[index]==1 and data_collection_train_browser[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_browser[index]==0 and data_collection_train_browser[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_browser[index]==0 and data_collection_train_browser[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_pred_Median_voice:
    if data_collection_train_pred_Median_voice[index]==1 and data_collection_train_voice[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_voice[index]==0 and data_collection_train_voice[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_voice[index]==0 and data_collection_train_voice[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0   
for i in data_collection_train_pred_Median_payments:
    if data_collection_train_pred_Median_payments[index]==1 and data_collection_train_payments[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_payments[index]==0 and data_collection_train_payments[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_payments[index]==0 and data_collection_train_payments[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_pred_Median_location:
    if data_collection_train_pred_Median_location[index]==1 and data_collection_train_location[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_location[index]==0 and data_collection_train_location[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_location[index]==0 and data_collection_train_location[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_pred_Median_IP:
    if data_collection_train_pred_Median_IP[index]==1 and data_collection_train_IP[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_IP[index]==0 and data_collection_train_IP[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_IP[index]==0 and data_collection_train_IP[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_collection_train_pred_Median_passward:
    if data_collection_train_pred_Median_passward[index]==1 and data_collection_train_passward[index]==1:
        TP +=1
    elif data_collection_train_pred_Median_passward[index]==0 and data_collection_train_passward[index]==0:
        TN +=1
    elif data_collection_train_pred_Median_passward[index]==0 and data_collection_train_passward[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1
    
recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 0.895910780669145
Precision: 0.7438271604938271
Accuracy: 0.7168367346938775
F1 Score: 0.8128161888701517


In [97]:
# ##Privacy Policy Change Matrix:

# ##Matrix creation:
# matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
#                                                'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
#             ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
#             ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
#            ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
#             ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
#             ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
# matrix_data_security_Comparison['Phrase'] = [['we save your personal data','we are not collect your personal data']
#                                  ,['we save your media','we are not collect your media']
#                                  ,['we save your browsing history and preferences','we are not collect your browsing history and preferences']
#                                  ,['we save your audio and recording','we are not collect your audio and recording']
#                                  ,['we save your payments data','we are not collect your payments data']
#                                  ,['we save your location and language','we are not collect your location and language']
#                                  ,['we save your IP address and browser and device details','we are not collect your IP address and browser and device details']
#                                  ,['we save your passwords','we are not collect your passwords']]

# ##Filling the matrix:
# websites_index = 0
# line = 0
# pred = 0
# data_security_train_pred_Comparison = []
# data_security_list_Comparison = []

# for j in sequence_to_classify:
#     for i in matrix_data_security_Comparison["Phrase"]:
#         a = classifier(str(j), str(i))
#         if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
#             pred = 1
#         else:
#             pred = 0 
#         if pred == 1:
#             data_security_list_Comparison.append(line+1)
#         line += 1
#     websites_index += 1
#     data_security_train_pred_Comparison.append(data_security_list_Comparison)
#     data_security_list_Comparison = []
#     line = 0

# data_security_train_pred_Comparison


[[1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [4, 5, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 4, 5, 6, 7, 8],
 [1, 2, 4],
 [1, 2, 3, 4, 5, 7, 8],
 [1, 2, 4, 5, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [4, 8],
 [1, 2, 7],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 7, 8],
 [2, 4, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 3, 4, 7, 8],
 [1, 3, 5, 7]]

In [67]:

##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your personal data','we are not collect your personal data']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_personal = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_personal.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_personal


[[1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1]]

In [68]:

##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your media','we are not collect your media']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_media = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_media.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_media

[[1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1]]

In [69]:
##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your browsing history and preferences','we are not collect your browsing history and preferences']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_browser = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_browser.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_browser

[[1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1]]

In [70]:
##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your audio and recording','we are not collect your audio and recording']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_voice = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_voice.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_voice

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [1]]

In [71]:
##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your payments data','we are not collect your payments data']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_payments = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_payments.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_payments

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [72]:
##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your location and language','we are not collect your location and language']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_lng = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_lng.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_lng

[[1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [1],
 [1],
 [1],
 [1]]

In [73]:
##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your IP address and browser and device details','we are not collect your IP address and browser and device details']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_IP = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_IP.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_IP

[[1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [],
 [1],
 [],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [124]:
##Matrix creation:
matrix_data_security_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_data_security_Comparison['Phrase'] = [['we save your passwords','we are not collect your passwords']]

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
data_security_train_pred_Comparison_password = []
data_security_list_Comparison = []

for j in sequence_to_classify:
    for i in matrix_data_security_Comparison["Phrase"]:
        a = classifier(str(j), str(i))
        if matrix_data_security_Comparison['Phrase'][line][0] in a["labels"][0]:
            pred = 1
        else:
            pred = 0 
        if pred == 1:
            data_security_list_Comparison.append(line+1)
        else:
            data_security_list_Comparison.append(line)
        line += 1
    websites_index += 1
    data_security_train_pred_Comparison_password.append(data_security_list_Comparison)
    data_security_list_Comparison = []
    line = 0

data_security_train_pred_Comparison_password

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1]]

In [134]:
data_collection_train_personal = [[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]
data_collection_train_media = [[1],[1],[0],[0],[1],[1],[0],[0],[1],[1],[1],[0],[0],[1],[0],[1],[1],[1],[0],[1],[0],[1],[1],[0],[0],[0],[1],[1],[0],[1],[1],[1],[1],[1],[0],[0],[1],[0],[1],[1],[1],[0],[0],[0],[1],[0],[1],[0],[0]]
data_collection_train_browser = [[1],[1],[1],[1],[1],[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]
data_collection_train_voice = [[1],[1],[0],[0],[1],[1],[1],[0],[0],[1],[0],[0],[0],[0],[0],[0],[1],[1],[0],[0],[0],[0],[1],[0],[0],[0],[1],[1],[0],[0],[0],[1],[0],[1],[1],[0],[1],[0],[1],[1],[0],[0],[0],[1],[0],[0],[1],[0],[0]]
data_collection_train_payments = [[1],[1],[1],[0],[0],[1],[1],[1],[1],[1],[1],[1],[0],[0],[0],[1],[1],[1],[1],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[0],[1],[1],[1],[0],[1],[1],[0],[1],[1],[1],[1]]
data_collection_train_location = [[1],[1],[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[0],[0],[0],[1],[1],[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[0],[1],[0],[1],[1],[0],[0],[0],[1],[1],[1],[1],[1],[1]]
data_collection_train_IP = [[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[0],[1],[1],[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[0],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]
data_collection_train_passward = [[1],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[1],[0],[0],[1],[0],[1],[0],[0],[1],[1],[1],[1],[0],[0],[0],[1],[0],[1],[0],[1],[0],[1],[1],[0],[1],[0],[1],[0]]
 

In [135]:
index = 0
for i in data_security_train_pred_Comparison_personal:
    if i==[]:
        data_security_train_pred_Comparison_personal[index]=[0]
    index +=1

    
index = 0
for i in data_security_train_pred_Comparison_media:
    if i==[]:
        data_security_train_pred_Comparison_media[index]=[0]
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_browser:
    if i==[]:
        data_security_train_pred_Comparison_browser[index]=[0]
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_voice:
    if i==[]:
        data_security_train_pred_Comparison_voice[index]=[0]
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_payments:
    if i==[]:
        data_security_train_pred_Comparison_payments[index]=[0]
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_lng:
    if i==[]:
        data_security_train_pred_Comparison_lng[index]=[0]
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_IP:
    if i==[]:
        data_security_train_pred_Comparison_IP[index]=[0]
    index +=1

In [143]:
#Data Collection Parameter:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in data_security_train_pred_Comparison_personal:
    if data_security_train_pred_Comparison_personal[index]==[1] and data_collection_train_personal[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_personal[index]==[0] and data_collection_train_personal[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_personal[index]==[0] and data_collection_train_personal[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_media:
    if data_security_train_pred_Comparison_media[index]==[1] and data_collection_train_media[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_media[index]==[0] and data_collection_train_media[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_media[index]==[0] and data_collection_train_media[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0
for i in data_security_train_pred_Comparison_browser:
    if data_security_train_pred_Comparison_browser[index]==[1] and data_collection_train_browser[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_browser[index]==[0] and data_collection_train_browser[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_browser[index]==[0] and data_collection_train_browser[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_security_train_pred_Comparison_voice:
    if data_security_train_pred_Comparison_voice[index]==[1] and data_collection_train_voice[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_voice[index]==[0] and data_collection_train_voice[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_voice[index]==[0] and data_collection_train_voice[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0   
for i in data_security_train_pred_Comparison_payments:
    if data_security_train_pred_Comparison_payments[index]==[1] and data_collection_train_payments[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_payments[index]==[0] and data_collection_train_payments[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_payments[index]==[0] and data_collection_train_payments[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_security_train_pred_Comparison_lng:
    if data_security_train_pred_Comparison_lng[index]==[1] and data_collection_train_location[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_lng[index]==[0] and data_collection_train_location[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_lng[index]==[0] and data_collection_train_location[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_security_train_pred_Comparison_IP:
    if data_security_train_pred_Comparison_IP[index]==[1] and data_collection_train_IP[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_IP[index]==[0] and data_collection_train_IP[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_IP[index]==[0] and data_collection_train_IP[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
index = 0    
for i in data_security_train_pred_Comparison_password:
    if data_security_train_pred_Comparison_password[index]==[1] and data_collection_train_passward[index]==[1]:
        TP +=1
    elif data_security_train_pred_Comparison_password[index]==[0] and data_collection_train_passward[index]==[0]:
        TN +=1
    elif data_security_train_pred_Comparison_password[index]==[0] and data_collection_train_passward[index]==[1]:
        FN +=1
    else:
        FP +=1
    index +=1
    
recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)


recall: 0.8392857142857143
Precision: 0.7078313253012049
Accuracy: 0.6377551020408163
F1 Score: 0.7679738562091504
